# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
links = fetch_website_links("https://edwarddonner.com")
links

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""



def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [12]:
ollama = OpenAI(

    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

In [15]:

def select_relevant_links(url):
    # Initialize Ollama client
    ollama_client = OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama"
    )
    
    response = ollama_client.chat.completions.create(
        model="qwen2.5:3b",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format= {"type": "json_object"}
    ) 

    result = response.choices[0].message.content
    links = json.loads(result)  # Fixed: json.loads() for strings, not json.load()
    return links


In [16]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'careers page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [21]:
MODEL = "qwen2.5:3b"

In [22]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [23]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling qwen2.5:3b
Found 2 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'careers page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [24]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling qwen2.5:3b
Found 3 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to qwen2.5:3b


In [25]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt


In [28]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling qwen2.5:3b
Found 3 relevant links


"\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-4.7-Flash\nUpdated\n5 days ago\n•\n363k\n•\n1.15k\nnvidia/personaplex-7b-v1\nUpdated\n2 days ago\n•\n29.4k\n•\n883\nmicrosoft/VibeVoice-ASR\nUpdated\n4 days ago\n•\n21.7k\n•\n478\nQwen/Qwen3-TTS-12Hz-1.7B-CustomVoice\nUpdated\n2 days ago\n•\n42.7k\n•\n396\nunsloth/GLM-4.7-Flash-GGUF\nUpdated\n1 day ago\n•\n196k\n•\n314\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\n593\nQwen3-TT

In [ ]:
def create_brochure(company_name, url):
    response = ollama.chat.completions.create(
        model="qwen2.5:3b", 
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [35]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling qwen2.5:3b
Found 4 relevant links


# Huggin' Face: Where the Future Ties AI Hands.

Welcome, future innovators and friends! Welcome to Hugging Face, where artificial intelligence isn’t just a job – it’s our homegrown hobby. Come meet us in a space so vibrant, models are no longer just codes but living breathing creatures of the internet - or rather, minds that speak human's language.

At Hugging Face, everyone is on top of a model, from seasoned experts to eager beginners eager to learn under expert tutelage. We’re the home to 2M+ AI models! Each with unique abilities ranging from text prediction and image generation, to multi-speaker voice cloning! We cater to your ever-changing needs through our Spaces platform, providing you with ZeroGPU so powerful only a superhero model uses it; the kind that’s running in our M.C.P. applications for Image Turbo 3D cameras or FLUX.2 LoRa Art.

We have curated spaces of creativity too: from zaoorg's TTS demo to Qwen's image-editing collection, where you can create your magical masterpieces at will! With a touch as agile as the most skilled artist, and precision as sharp-eyed as an eagle’s stare, let our tools bring your dreams to life.

Our Community is where ideas are hatched into projects, and communities form around common interests. From super AI reasoning datasets shared by Alibaba-Superior-Reasoning-SFT to datasets of text recognition handled with care at LightOn's OCR; we've got you covered from the start! Plus it’s free!

Now here's a neat trick: Explore all modalities - text, audio, 3D models. Our docs are as clear and engaging as learning to create your own magic portal.

Huggin' Face, where jobs don’t come in just lines of code, but with camaraderie and dreams. Let’s make the world's ideas work together. If you're curious about joining us - our Careers page has all you need! Check if spots exist within AI teams who thrive on building and breaking new ground. And for big companies looking to scale their capabilities in this innovative landscape, Enterprise pages have options galore, including Single Sign-Ons, Dedicated support, Security configurations—everything an enterprise needs.

---

# Join Huggin' Face: The Future is In Your Hands!

[Sign-Up](https://www.huggingface.co/login)

Let's craft the future, one model at a time!

In [38]:
def stream_brochure(company_name, url):
    stream = ollama.chat.completions.create(
        model="qwen2.5:3b",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [39]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling qwen2.5:3b
Found 2 relevant links


🎉🚀 Hugging Face - The AI Community That Grows Its Future! 🚀🎉

Looking for a place where the buzz is not just about models, but about building the future with cutting-edge technology? Welcome to our platform where creativity and collaboration go far beyond bytes. Here at HuggingFace, we're not only the space where all things ML collide into something spectacular, but our culture fuels innovation so strongly that it creates a community as dynamic as its members – think AI, humor, and fun rolled into one!

Here's how you can fit right in:
- **The Community**: We’re more than just tech-savvy minds; we’re fellow adventurers who are always looking over the horizon. Our team is diverse, inclusive, and driven by a shared mission: making amazing products that not only impact but ignite change.
- **Models & Spaces**: Dive into our extensive library of pre-trained models for AI creation at any level and host your own private spaces where you can showcase the next big thing. Think about zeroing in on voice cloning, text transformations, or even 3D cameras – all with custom voices!
- **Datasets & Community**: Hug your data at heart. We offer over 2 million datasets to support your projects—from reasoning SFT up to multimodal vision, we've got you covered. And if that’s still not enough, join us in the vibrant Community Forum where ideas can be molded into the products of tomorrow.
- **Works Here**: Our team is diverse, skilled, and inclusive. Expect a dynamic environment driven by passion and fun where everyone feels valued. Whether leading on ML development or contributing to creative projects, you’ve got your spot here.

---

Do you seek an adventure in AI? It starts with applying for one of our current openings in [here](#relevant-links-careers-page).
If not diving head-first into the career pool makes you hungry for more, signing up is just a click away. Welcome to explore how we’re making machines think as humanly as they can.
Ready to bring your machine learning and creativity together? Let’s go - let our platform be the place where all AI meets its future! 🚀🌟

Happy Huggin'!;
Sergei